Importing dependies

In [ ]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

loading the dataset and preview it

In [ ]:
movie_df=pd.read_csv('TMDB_new_dataset.csv')

In [ ]:
movie_df.head()

In [ ]:
movie_df.shape

In [ ]:
def preprocess_text(text):
    # Lowercase and remove non-alphanumeric characters
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text.strip()

Selcting the relevent features

In [ ]:
selected_features = ['keywords', 'genres', 'title','production_companies','tagline']
for feature in selected_features:
    movie_df[feature] = movie_df[feature].fillna('')
    movie_df[feature] = movie_df[feature].apply(preprocess_text)

Combining the all relevant features

In [ ]:
movie_df['combined_features'] = movie_df['keywords'] + ' ' + movie_df['genres'] + ' ' + movie_df['title']+ ' ' + movie_df['production_companies'] + ' ' + movie_df['tagline']

Converting text data to feature vectors

In [ ]:
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(movie_df['combined_features'])

In [ ]:
print(feature_vectors)

Using cosine_similarity algo for finding similarity

In [ ]:
similarity = cosine_similarity(feature_vectors)

Exporting the saved and modified dataset

In [ ]:
movie_df.to_csv('../../TMDB_model.csv', index=False)

Saving the trained model

In [ ]:
# import pickle

# filename='trained_TMDB.sav'
# pickle.dump(similarity,open(filename,'wb'))